Part 2: Depth from Rectified Stereo Images

Write a program to estimate a depth map from the pair of rectified images in Figure 2 using an MRF and multiple label graphcuts.
Show your best result and provide some discussion if necessary (particularly if the results are not as good as expected).

In [1]:

import numpy
import statistics
import cv2
from functools import cache
from gco import pygco
from utils import get_logger, INTEGER_MAX_VALUE
import matplotlib.image
import matplotlib.pyplot
import queue
import typing
import gc

/Users/admin/EE5371/CA2/.venv/lib/python3.10/site-packages/gco/__init__.py:7: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, name):


In [2]:
# @cache
def compute_distance(point1: numpy.ndarray, point2: numpy.ndarray) -> float:
    """
    compute distance of two pixels
    Parameters
    ----------
    point1: ndarray, uint8, shape=(3)
        point one
    point2: ndarray, uint8, shape=(3)
        point two
    """
    # return (abs(point1[0]-point2[0]) + abs(point1[1] - point2[1]) + abs(point1[2] - point2[2]) ) / 3 / 255
    # if not dividing 255 as the C pseudocode example, there will be a overflowing of MAX_VALUE of Integer when calling pycgo.cut_grid_graph
    return numpy.sum(numpy.abs(point1-point2)) /3
def linear_normalize(data: numpy.ndarray) -> numpy.ndarray:
    """
        linear normalize
    """
    min_value = data.min()
    max_value = data.max()
    normalized_data = (data - min_value) / (max_value - min_value)
    return normalized_data

def normal_distribution_normalize(data: numpy.ndarray) -> numpy.ndarray:
    """
        normalize by normal distribution
    """
    mean = numpy.mean(data)
    std = numpy.std(data)
    normalized_data = (data - mean) / std
    return normalized_data


In [3]:
img1: numpy.ndarray = cv2.imread("data/rectified_image_1.png", cv2.IMREAD_COLOR)
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
height1, width1, _ = img1.shape

img2: numpy.ndarray = cv2.imread("data/rectified_image_2.png", cv2.IMREAD_COLOR)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img2 = cv2.resize(img2, (width1, height1))

height2, width2, _ = img2.shape
row_list: [INTEGER_MAX_VALUE] * width2
disparity_by_y: typing.List[int] = []
disparity_by_x: typing.List[int] = []
for y in range(height1):
    # img2: still y
    disparity_by_x.clear()
    for x1 in range(width1):
        pq = queue.PriorityQueue()
        for x2 in range(width2):
            pixel1 = img1[y, x1]
            pixel2 = img2[y, x2]
            dis = compute_distance(point1=pixel1, point2=pixel2)
            pq.put(item=(dis, x2))
        x2_argmin = pq.get()[1]
        disparity = abs(x2_argmin - x1)
        disparity_by_x.append(disparity)
    mode = statistics.mode(disparity_by_x)
    disparity_by_y.append(mode)
disparity_by_x.clear()

In [4]:
print(f"disparity_by_y:{disparity_by_y}")
tem = img1
img1 = img2
img2 = tem
depth_min = min(disparity_by_y)
depth_max = max(disparity_by_y)
depth_max= 64
# todo: change to list
# STEPS:typing.List[int] = [20,15, 5,6,7,8,9,10]
STEPS:typing.List[int] = [50]
# STEPS:typing.List[int] = [5,6,7]
for STEP in STEPS:
# STEP = 5
    depth_range = (depth_max - depth_min) // STEP
    print(f"depth_range:{depth_range}")
    unary_cost: numpy.ndarray = numpy.zeros(shape=(height1, width1, STEP), dtype=numpy.float64)
    for y in range(height1):
        for x in range(width1):
            x2_start = min(width1 - 1, x + depth_min)
            x2_end = min(width1, x2_start + depth_range)
            for i in range(STEP):
                img2_matched = img2[y, x2_start: x2_end]
                x2_start = min(width1 - 1, x2_start + depth_range)
                x2_end = min(width1, x2_start + depth_range)
                disparity_by_x.clear()
                for pixel2 in img2_matched:
                    disparity_by_x.append(compute_distance(point1=img1[y,x], point2=pixel2))
                unary_cost[y, x, i] = min(disparity_by_x) /3 / 255
    # unary_cost = linear_normalize(data=unary_cost)
    unary_cost *= depth_range
    # print(f"unary_cost:{unary_cost}")
    unary_cost = unary_cost[:, : width1-depth_max, :]
    i = numpy.arange(STEP, dtype=numpy.int32)
    j = numpy.arange(STEP, dtype=numpy.int32)
    fp = numpy.abs(i[: , numpy.newaxis] - j)
    # LAMBDAS = [0.005, 0.006, 0.007, 0.01,0.02,0.03, 0.04, 0.05, 0.06,0.07,0.08,0.09,0.10]
    LAMBDAS = [0.001, 0.002, 0.003, 0.004,0.005, 0.01, 0.02, 0.03, 0.04, 0.045, 0.05, 0.055, 0.06,0.07,0.08,0.09,0.1]
    lam = 0.03
    # for lam in LAMBDAS:
    # while lam < 0.061:
    for lam in LAMBDAS:
        gc.collect()
        # print(f"fp:{fp}")
        labels = pygco.cut_grid_graph_simple(unary_cost=unary_cost, pairwise_cost=fp * lam, connect=8, algorithm='swap')
        labels = labels.reshape(unary_cost.shape[0], unary_cost.shape[1])
        labels = labels[:,depth_range:]
        mapped_array = ((labels) / (depth_range - 1)) * (255)
        # print(f"labels:{labels}")
        # labels = cv2.bilateralFilter(numpy.float32(labels), 3, 75, 75)
        img_path = f"data/labels{lam}-{STEP}.png"
        labels = cv2.convertScaleAbs(src=labels, alpha=5, beta=0)
        cv2.imwrite(img_path, labels)
        img_show = matplotlib.image.imread(img_path)
        matplotlib.pyplot.title(img_path)
        matplotlib.pyplot.imshow(X=img_show, cmap='gray') 
        matplotlib.pyplot.show()
        # lam += 0.001

disparity_by_y:[20, 19, 19, 18, 21, 20, 20, 19, 20, 17, 18, 18, 21, 18, 19, 20, 21, 20, 20, 20, 18, 19, 20, 18, 15, 20, 19, 20, 18, 24, 20, 20, 18, 19, 16, 20, 18, 20, 19, 19, 20, 20, 21, 20, 20, 19, 21, 18, 20, 21, 18, 22, 23, 23, 23, 21, 23, 21, 19, 20, 21, 22, 22, 20, 22, 22, 22, 21, 20, 19, 20, 19, 19, 19, 19, 22, 21, 19, 21, 21, 20, 20, 20, 22, 20, 22, 19, 21, 20, 22, 23, 18, 24, 19, 18, 22, 19, 20, 25, 25, 20, 23, 22, 18, 27, 18, 19, 20, 20, 19, 25, 23, 20, 22, 19, 26, 26, 25, 25, 20, 25, 19, 20, 20, 20, 23, 20, 24, 25, 18, 18, 29, 32, 31, 25, 30, 32, 30, 28, 32, 31, 21, 30, 39, 29, 29, 29, 27, 31, 23, 31, 27, 27, 29, 32, 27, 27, 31, 26, 30, 31, 30, 33, 32, 32, 29, 30, 32, 28, 28, 31, 30, 32, 30, 30, 28, 31, 30, 30, 30, 32, 33, 32, 29, 31, 30, 33, 29, 30, 30, 30, 35, 32, 30, 30, 30, 28, 27, 33, 32, 29, 32, 43, 40, 33, 33, 29, 32, 42, 38, 30, 34, 28, 30, 25, 32, 33, 36, 30, 30, 31, 33, 32, 29, 44, 34, 49, 37, 31, 28, 37, 34, 34, 37, 46, 43, 36, 39, 39, 40, 40, 38, 42, 39, 38, 36, 

ValueError: min() arg is an empty sequence